# 04b - Multiclass Federated Learning with Edge IIoT Dataset using Flower and TensorFlow/Keras

In this notebook we use the Flower Federated Learning library (flower.dev) with Tensorflow/Keras to distribute the Edge-IIoT data across multiple clients in various different ways.

In [1]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['INDIVIDUAL_ATTACK', 'ATTACK_GROUP', 'STRATIFIED']
METHOD = 'STRATIFIED'
NUM_OF_STRATIFIED_CLIENTS = 10 # only applies to stratified method
print (METHOD)

STRATIFIED


In [2]:
%%capture
%pip install flwr[simulation] torch torchvision matplotlib sklearn openml

In [3]:
import os
import pandas as pd
import numpy as np
import flwr as fl
import sklearn
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10


In [4]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.3.0
numpy 1.24.2
torch 1.13.1
torchvision 0.14.1
Training on cuda:0


In [5]:
dataset_path = "../datasets/Edge-IIoT/"

df = pd.read_pickle(dataset_path + "Edge-IIoTset dataset/Selected dataset for ML and DL/ML-EdgeIIoT-dataset.pkl")   

## Make dataframe for Multi-class classification

In [6]:
# Multiclass attack dataframe
multiclass_df = df[['frame.time', 'ip.src_host', 'ip.dst_host', 'arp.dst.proto_ipv4',
       'arp.opcode', 'arp.hw.size', 'arp.src.proto_ipv4', 'icmp.checksum',
       'icmp.seq_le', 'icmp.transmit_timestamp', 'icmp.unused',
       'http.file_data', 'http.content_length', 'http.request.uri.query',
       'http.request.method', 'http.referer', 'http.request.full_uri',
       'http.request.version', 'http.response', 'http.tls_port', 'tcp.ack',
       'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin',
       'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack',
       'tcp.dstport', 'tcp.flags', 'tcp.flags.ack', 'tcp.len', 'tcp.options',
       'tcp.payload', 'tcp.seq', 'tcp.srcport', 'udp.port', 'udp.stream',
       'udp.time_delta', 'dns.qry.name', 'dns.qry.name.len', 'dns.qry.qu',
       'dns.qry.type', 'dns.retransmission', 'dns.retransmit_request',
       'dns.retransmit_request_in', 'mqtt.conack.flags',
       'mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.hdrflags', 'mqtt.len',
       'mqtt.msg_decoded_as', 'mqtt.msg', 'mqtt.msgtype', 'mqtt.proto_len',
       'mqtt.protoname', 'mqtt.topic', 'mqtt.topic_len', 'mqtt.ver',
       'mbtcp.len', 'mbtcp.trans_id', 'mbtcp.unit_id',
       'Attack_type']]

In [7]:
known_sensor_ip_addresses = [ '192.168.0.101', '192.168.2.194', '192.168.3.18', '192.168.4.73', '192.168.5.47', '192.168.6.56', '192.768.7.62', '192.168.8.163']
print ("known_sensor_ip_addresses:", known_sensor_ip_addresses)

tcp_dos_attack_ip_addresses = [ '207.192.25.133', '94.196.109.185', '133.149.252.77', '220.146.94.148' ]
tdp_dos_atack_ip_addresses = [ '190.123.219.128', '16.226.184.201', '153.125.214.15', '91.184.12.91' ]
http_attack_ip_addresses = [ '192.168.0.170', '216.58.198.74' ]
icmp_flood_attack_ip_addresses = [ '213.117.18.213', '183.223.100.122', '166.153.227.121', '49.81.59.152', '227.117.33.125' ]
port_scan_attack_ip_addresses = [ '192.168.0.170' ]
os_fingerprinting_attack_ip_addresses = [ '192.168.0.170' ]
vuln_scan_attack_ip_addresses = [ '192.168.0.170', '142.250.200.205', '172.217.19.35', '142.250.201.10' ]
dns_spoof_attack_ip_addresses = [ '192.168.0.101', '192.168.0.152', '172.217.19.35', '192.168.0.170' ]
arp_spoof_attack_ip_addresses = [ '192.168.0.101', '192.168.0.152', '172.217.19.35', '192.168.0.170' ]
xss_attack_ip_addresses = [ '192.168.0.170', '172.217.19.42', '104.16.87.20' ]
sql_injection_attack_ip_addresses = [ '192.168.0.170' ]
upload_attack_ip_addresses = [ '192.168.0.170' ]
backdoor_attack_ip_addresses = [ '192.168.0.170' ]
password_attack_ip_addresses = [ '192.168.0.170' ]
ransomware_attack_ip_addresses = [ '192.168.0.170' ] 

# Combine all attack IP addresses into one list, ensuring no duplicates
known_attacker_ip_addresses = list(set(tcp_dos_attack_ip_addresses + tdp_dos_atack_ip_addresses + http_attack_ip_addresses + icmp_flood_attack_ip_addresses + port_scan_attack_ip_addresses + os_fingerprinting_attack_ip_addresses + vuln_scan_attack_ip_addresses + dns_spoof_attack_ip_addresses + arp_spoof_attack_ip_addresses + xss_attack_ip_addresses + sql_injection_attack_ip_addresses + upload_attack_ip_addresses + backdoor_attack_ip_addresses + password_attack_ip_addresses + ransomware_attack_ip_addresses))
print (f"known_attacker_ip_addresses: \nNumber of IPs {len(known_attacker_ip_addresses)}\n{known_attacker_ip_addresses}")


known_sensor_ip_addresses: ['192.168.0.101', '192.168.2.194', '192.168.3.18', '192.168.4.73', '192.168.5.47', '192.168.6.56', '192.768.7.62', '192.168.8.163']
known_attacker_ip_addresses: 
Number of IPs 22
['207.192.25.133', '153.125.214.15', '192.168.0.152', '142.250.201.10', '220.146.94.148', '91.184.12.91', '192.168.0.101', '183.223.100.122', '94.196.109.185', '166.153.227.121', '133.149.252.77', '172.217.19.35', '104.16.87.20', '142.250.200.205', '172.217.19.42', '190.123.219.128', '227.117.33.125', '216.58.198.74', '213.117.18.213', '192.168.0.170', '16.226.184.201', '49.81.59.152']


With the exception of DDOS attacks, the attacks mainly come from a small subset of IP addresses. `192.168.0.170` being responsible for a lot of the attacks. From the data exploration workbook `02b-ML-Data-Exploration.ipynb` we can also see that the attacked IP is always the `192.168.0.128` edge server. This means it is not feasible the divide the traffic either by attacker or attacked IP address.

## Multiclass Classification

Categorical data encoding (Dummy Encoding):

EG. Takes a product category and converts it to a binary vector

In [8]:
def encode_text_dummy(df, name):

    dummies = pd.get_dummies(df[name])

    for x in dummies.columns:

        dummy_name = f"{name}-{x}"

        df[dummy_name] = dummies[x]

    df.drop(name, axis=1, inplace=True)

encode_text_dummy(multiclass_df,'http.request.method')

encode_text_dummy(multiclass_df,'http.referer')

encode_text_dummy(multiclass_df,"http.request.version")

encode_text_dummy(multiclass_df,"dns.qry.name.len")

encode_text_dummy(multiclass_df,"mqtt.conack.flags")

encode_text_dummy(multiclass_df,"mqtt.protoname")

encode_text_dummy(multiclass_df,"mqtt.topic")

In [9]:
# print max index value of binary_df
print("max index value of binary_df:", max(multiclass_df.index))
print(multiclass_df.shape)

max index value of binary_df: 157799
(157800, 90)


We need to drop some unrequired columns from the DF, but we need to keep the original around as we may need to split the data differently for different models based on things like the IP address

In [10]:
#drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 
drop_columns = ["frame.time", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 

         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",

         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",

         "tcp.dstport", "udp.port", "mqtt.msg"]

multiclass_df = multiclass_df.drop(drop_columns, axis=1)

multiclass_df = multiclass_df.dropna(axis=0, how='any')

multiclass_df = multiclass_df.drop_duplicates(subset=None, keep="first")

# We cant shuffle at this point as we need to keep the order so we can split the dataset later based on things like IP address
#binary_df_copy = shuffle(binary_df_copy)

# Compute the number of missing values (NaN or null) in each column of a pandas DataFrame object named df.
multiclass_df.isna().sum()

ip.src_host                            0
ip.dst_host                            0
arp.opcode                             0
arp.hw.size                            0
icmp.checksum                          0
                                      ..
mqtt.protoname-0.0                     0
mqtt.protoname-MQTT                    0
mqtt.topic-0                           0
mqtt.topic-0.0                         0
mqtt.topic-Temperature_and_Humidity    0
Length: 77, dtype: int64

In [11]:
# print max index value of binary_df
print("max index value of binary_df:", max(multiclass_df.index))
print(multiclass_df.shape)

multiclass_df.reset_index(drop=True, inplace=True)

# print max index value of binary_df
print("max index value of binary_df:", max(multiclass_df.index))
print(multiclass_df.shape)


max index value of binary_df: 157799
(153225, 77)
max index value of binary_df: 153224
(153225, 77)


Change the Attack Type to be a unique number for the attack type

In [12]:
# Creating a dictionary of Types
attacks = {'Normal': 0 ,'Backdoor' :1, 'DDoS_HTTP':2,  'DDoS_ICMP':3, 'DDoS_TCP':4, 'DDoS_UDP':5, 
           'Fingerprinting':6, 'MITM':7, 'Password':8, 'Port_Scanning':9, 'Ransomware':10, 
           'SQL_injection':11, 'Uploading':12, 'Vulnerability_scanner':13, 'XSS':14}

multiclass_df['Attack_type'] = multiclass_df['Attack_type'].map(attacks)


In [13]:
label = multiclass_df['Attack_type']
le = preprocessing.LabelEncoder()
label_n = le.fit_transform(label.values)

# Stratify based on the attack label to balance the dataset - This is our original copy of the data include IP addresses
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(multiclass_df, label_n, stratify=label_n, test_size=0.2, random_state=42)

print(X_train_df.shape)

X_train_df.reset_index(drop=True, inplace=True)

#print the max index of X_train_df
print(X_train_df.index.max())

#  print the max index of multiclass_df
print(multiclass_df.index.max())

(122580, 77)
122579
153224


In [14]:
print(label_n)

[7 7 7 ... 3 3 3]


In [15]:
multiclass_df_copy = multiclass_df.copy()

multiclass_df_copy = multiclass_df_copy.drop(["ip.src_host", "ip.dst_host", "Attack_type"], axis=1)

# This is our copy of the data without IP addresses
scaled_features = StandardScaler().fit_transform(multiclass_df_copy.values)
X_train, X_test, y_train, y_test = train_test_split(scaled_features, label_n, stratify=label_n, test_size=0.2, random_state=42)

print ("Train:", X_train.shape, y_train.shape)
print ("Test:", X_test.shape, y_test.shape)


Train: (122580, 74) (122580,)
Test: (30645, 74) (30645,)


In [16]:
fl_X_train = []
fl_y_train = []

if METHOD == 'STRATIFIED':
    # Stratfiy the dataset
    from sklearn.model_selection import StratifiedKFold

    skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)
    skf.get_n_splits(X_train, y_train)

    for _, train_index in skf.split(X_train, y_train):
        print("TRAIN:", train_index)
        X_np = X_train[train_index]
        y_np = y_train[train_index]

        fl_X_train.append(X_np)
        fl_y_train.append(y_np)

else: # UNUSED
    # Individual IP address
    for ip in known_sensor_ip_addresses:
        new_ip = [ip]
        print("new_ip:", new_ip)

        X_train_df['ip.src_host']
        
        print("Shape X_Train:", X_train.shape)
        print("Shape y_Train:", y_train.shape)

        # Filter dataframe by IP address
        new_df_src = X_train_df[ X_train_df['ip.src_host'].isin(new_ip) ]
        new_df_dst = X_train_df[ X_train_df['ip.dst_host'].isin(new_ip) ]

        print("Shape new_df_src:", new_df_src.shape)
        print("Shape new_df_dst:", new_df_dst.shape)

        X_np = np.vstack([ X_train[ new_df_src.index, : ], X_train[ new_df_dst.index, :] ])
        y_np = np.hstack([ y_train[ new_df_src.index ], y_train[ new_df_dst.index ] ])

        print ("x_np:", X_np.shape)
        print ("y_np:", y_np.shape)

        fl_X_train.append(X_np)
        fl_y_train.append(y_np)


TRAIN: [     0     19     25 ... 122549 122557 122567]
TRAIN: [     2     13     21 ... 122544 122568 122573]
TRAIN: [     6     12     20 ... 122574 122576 122577]
TRAIN: [    35     56     73 ... 122554 122565 122570]
TRAIN: [     3      4      5 ... 122536 122572 122579]
TRAIN: [     7     11     14 ... 122533 122563 122575]
TRAIN: [    10     50     61 ... 122561 122564 122571]
TRAIN: [     9     18     24 ... 122550 122555 122560]
TRAIN: [     1     15     26 ... 122547 122562 122569]
TRAIN: [    17     29     32 ... 122559 122566 122578]


In [17]:
# Print out the size of the fl_X_train
print ("fl_X_train size:", len(fl_X_train))

# Print out the size of each element in the fl_X_train
for i in range(len(fl_X_train)):
    print ("fl_X_train[", i, "]:", fl_X_train[i].shape)
    print ("fl_y_train[", i, "]:", fl_y_train[i].shape)


fl_X_train size: 10
fl_X_train[ 0 ]: (12258, 74)
fl_y_train[ 0 ]: (12258,)
fl_X_train[ 1 ]: (12258, 74)
fl_y_train[ 1 ]: (12258,)
fl_X_train[ 2 ]: (12258, 74)
fl_y_train[ 2 ]: (12258,)
fl_X_train[ 3 ]: (12258, 74)
fl_y_train[ 3 ]: (12258,)
fl_X_train[ 4 ]: (12258, 74)
fl_y_train[ 4 ]: (12258,)
fl_X_train[ 5 ]: (12258, 74)
fl_y_train[ 5 ]: (12258,)
fl_X_train[ 6 ]: (12258, 74)
fl_y_train[ 6 ]: (12258,)
fl_X_train[ 7 ]: (12258, 74)
fl_y_train[ 7 ]: (12258,)
fl_X_train[ 8 ]: (12258, 74)
fl_y_train[ 8 ]: (12258,)
fl_X_train[ 9 ]: (12258, 74)
fl_y_train[ 9 ]: (12258,)


In [18]:
len(label.unique()) 

15

In [19]:
NUM_OF_CLIENTS = len(fl_X_train)
print("NUM_OF_CLIENTS:", NUM_OF_CLIENTS)    

NUM_OF_ROUNDS = 10

print("Checking data split groups")
for i in range(len(fl_X_train)):
    print(i, ":", "X Shape", fl_X_train[i].shape, "Y Shape", fl_y_train[i].shape)

print("\nDeploy Simulation")


NUM_OF_CLIENTS: 10
Checking data split groups
0 : X Shape (12258, 74) Y Shape (12258,)
1 : X Shape (12258, 74) Y Shape (12258,)
2 : X Shape (12258, 74) Y Shape (12258,)
3 : X Shape (12258, 74) Y Shape (12258,)
4 : X Shape (12258, 74) Y Shape (12258,)
5 : X Shape (12258, 74) Y Shape (12258,)
6 : X Shape (12258, 74) Y Shape (12258,)
7 : X Shape (12258, 74) Y Shape (12258,)
8 : X Shape (12258, 74) Y Shape (12258,)
9 : X Shape (12258, 74) Y Shape (12258,)

Deploy Simulation


# FL Part

NOTE TO SELF - BUILD IN F1 SCORE  - https://www.kaggle.com/code/gpiosenka/flower-classification-f1-score-93


In [54]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

print('scikit-learn {}.'.format(sklearn.__version__))
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout


class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=10, batch_size=64)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels, batch_size=64)
        print ("Client ", self.cid, "Evaluating complete...", accuracy, loss)
        return loss, len(self.train_data), {"accuracy": accuracy}
    
    def predict(self, incoming):
        prediction = np.argmax( self.model.predict(incoming) ,axis=1)
        return prediction

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(50, activation='relu'),  
      Dense(25, activation='relu'),  
      Dense(len(label.unique()), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

   
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)


print ("Deploy simulation...")

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
eval_count = 0

def get_evaluate_fn(server_model):
    global eval_count
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    
    
    def evaluate(server_round, parameters, config):
        global eval_count
        
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print ("Server Evaluating...", eval_count)
        loss, accuracy = server_model.evaluate(X_test, y_test)
        
        y_pred = server_model.predict(X_test)
        print ("Prediction: ", y_pred, y_pred.shape)
        #cmatrix = confusion_matrix(y_test, np.rint(y_pred))
        #print ("confusion_matrix:", cmatrix, cmatrix.shape)
                        
        print ("Server Evaluating complete...", accuracy, loss)
        
        np.save("y_pred-" + str(eval_count) + ".npy", y_pred)
        #np.save("cmatrix-" + str(eval_count) + ".npy", cmatrix)
        eval_count = eval_count + 1
        
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
    #Flatten(input_shape=(79,1)),
    Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
    Dense(50, activation='relu'),  
    Dense(25, activation='relu'),  
    Dense(len(label.unique()), activation='softmax')
])


server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

INFO flwr 2023-07-06 18:48:16,430 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


scikit-learn 1.2.0.
flwr 1.4.0
numpy 1.24.2
tf 2.11.0
Deploy simulation...
(launch_and_fit pid=36136) 24/24 [==============================] - ETA: 0s - loss: 0.3951 - accuracy: 0.8100
(launch_and_fit pid=43488) 16/24 [===================>..........] - ETA: 0s - loss: 0.3912 - accuracy: 0.8123 [repeated 11x across cluster]
(launch_and_fit pid=37272) 15/24 [=================>............] - ETA: 0s - loss: 0.4044 - accuracy: 0.8085 [repeated 6x across cluster]
(launch_and_evaluate pid=43488) Client ID: 4 [repeated 5x across cluster]
(launch_and_fit pid=29552) Epoch 10/10 [repeated 53x across cluster]
(launch_and_evaluate pid=11820)  [repeated 160x across cluster]
(launch_and_evaluate pid=11820)  1/24 [>.............................] - ETA: 3s - loss: 0.3740 - accuracy: 0.8047 [repeated 58x across cluster]
(launch_and_evaluate pid=11820) 24/24 [==============================] - 0s 1ms/step - loss: 0.3851 - accuracy: 0.8235 [repeated 5x across cluster]
(launch_and_evaluate pid=25520) Clie

2023-07-06 18:48:25,415	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-07-06 18:48:30,234 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 6719447040.0, 'object_store_memory': 3359723520.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0, 'CPU': 24.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 6719447040.0, 'object_store_memory': 3359723520.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0, 'CPU': 24.0}
INFO flwr 2023-07-06 18:48:30,237 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-07-06 18:48:30,238 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
INFO flwr 2023-07-06 18:48:34,794 | server.py:277 | Received initial parameters from one random client
INFO:flwr:Received initial parameters from one random client
INFO flwr 2023-07-06 18:48:34,796 | server.py:88 | Evaluating initial parameters
INFO:flwr:Evaluating

Server Evaluating... 0
(launch_and_get_parameters pid=38200) Client ID: 7
958/958 [==============================] - 1s 793us/step


INFO flwr 2023-07-06 18:48:36,911 | server.py:91 | initial parameters (loss, other metrics): 2.7574028968811035, {'accuracy': 0.056126609444618225}
INFO:flwr:initial parameters (loss, other metrics): 2.7574028968811035, {'accuracy': 0.056126609444618225}
INFO flwr 2023-07-06 18:48:36,913 | server.py:101 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2023-07-06 18:48:36,914 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)
DEBUG:flwr:fit_round 1: strategy sampled 10 clients (out of 10)


...

Prediction:  [[2.2826885e-07 2.2756515e-06 5.9995097e-07 ... 2.3113169e-01
  7.9292977e-07 9.1562288e-06]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 4.9580704e-12
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 9.6321007e-17 5.5370062e-27 ... 9.8976194e-10
  8.3224826e-24 1.8158216e-31]
 ...
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.5424488e-08 3.9670625e-04 3.4636075e-11 ... 1.7385064e-09
  8.0217936e-11 1.9537913e-12]
 [8.3317858e-08 2.2541126e-06 4.2989959e-06 ... 2.5739041e-01
  1.1306423e-05 2.4969152e-06]] (30645, 15)
Server Evaluating complete... 0.823625385761261 0.36255672574043274
(launch_and_evaluate pid=45420)  48/192 [======>.......................] - ETA: 0s - loss: 0.3570 - accuracy: 0.8262  [repeated 4x across cluster]
(launch_and_fit pid=25584) 105/192 [===============>..............] - ETA: 0s - loss: 0.3429 - accuracy: 0.8275 [repeated 3x across cluster]


DEBUG flwr 2023-07-06 18:49:49,278 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 5 results and 0 failures
INFO flwr 2023-07-06 18:49:49,280 | server.py:147 | FL finished in 72.36596099999588
INFO:flwr:FL finished in 72.36596099999588
INFO flwr 2023-07-06 18:49:49,281 | app.py:218 | app_fit: losses_distributed [(1, 0.5157408833503723), (2, 0.4572304666042328), (3, 0.40484787821769713), (4, 0.37182614803314207), (5, 0.3628541588783264), (6, 0.362065190076828), (7, 0.3600628852844238), (8, 0.353603732585907), (9, 0.35691051483154296), (10, 0.3564058244228363)]
INFO:flwr:app_fit: losses_distributed [(1, 0.5157408833503723), (2, 0.4572304666042328), (3, 0.40484787821769713), (4, 0.37182614803314207), (5, 0.3628541588783264), (6, 0.362065190076828), (7, 0.3600628852844238), (8, 0.353603732585907), (9, 0.35691051483154296), (10, 0.3564058244228363)]
INFO flwr 2023-07-06 18:49:49,282 | app.py:219 | app_fit: metrics_distributed_fit {

(launch_and_evaluate pid=45812) Client  4 Evaluating... [repeated 5x across cluster]
(launch_and_evaluate pid=45812) Client ID: 4 [repeated 5x across cluster]
(launch_and_fit pid=33324) Epoch 10/10 [repeated 90x across cluster]
(launch_and_evaluate pid=25464)  [repeated 659x across cluster]
(launch_and_evaluate pid=25464)   1/192 [..............................] - ETA: 32s - loss: 0.4026 - accuracy: 0.8438 [repeated 91x across cluster]
(launch_and_fit pid=33324)  34/192 [====>.........................] - ETA: 0s - loss: 0.3459 - accuracy: 0.8267 [repeated 65x across cluster]
(launch_and_fit pid=33324)  67/192 [=========>....................] - ETA: 0s - loss: 0.3442 - accuracy: 0.8258 [repeated 53x across cluster]
(launch_and_fit pid=33324)  87/192 [============>.................] - ETA: 0s - loss: 0.3419 - accuracy: 0.8306 [repeated 16x across cluster]
(launch_and_fit pid=33324) 117/192 [=================>............] - ETA: 0s - loss: 0.3461 - accuracy: 0.8279 [repeated 21x across c

History (loss, distributed):
	round 1: 0.5157408833503723
	round 2: 0.4572304666042328
	round 3: 0.40484787821769713
	round 4: 0.37182614803314207
	round 5: 0.3628541588783264
	round 6: 0.362065190076828
	round 7: 0.3600628852844238
	round 8: 0.353603732585907
	round 9: 0.35691051483154296
	round 10: 0.3564058244228363
History (loss, centralized):
	round 0: 2.7574028968811035
	round 1: 0.5143721699714661
	round 2: 0.4589138627052307
	round 3: 0.40377622842788696
	round 4: 0.376365602016449
	round 5: 0.36888301372528076
	round 6: 0.365296334028244
	round 7: 0.3644757866859436
	round 8: 0.36345842480659485
	round 9: 0.3633269965648651
	round 10: 0.36255672574043274
History (metrics, centralized):
{'accuracy': [(0, 0.056126609444618225), (1, 0.7136237621307373), (2, 0.7856420278549194), (3, 0.8178169131278992), (4, 0.819840133190155), (5, 0.8222548365592957), (6, 0.8212432861328125), (7, 0.821634829044342), (8, 0.8197095990180969), (9, 0.8219611644744873), (10, 0.823625385761261)]}

(launch_and_evaluate pid=25464) Client  1 Evaluating complete... 0.8281938433647156 0.38641098141670227
(launch_and_fit pid=25584) 174/192 [==========================>...] - ETA: 0s - loss: 0.3435 - accuracy: 0.8279 [repeated 13x across cluster]
